# GHZ circuit score comparison experiments

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import random

import cirq
import cirq_google as cg
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx 

from qbitmap import metrics
from qbitmap import utils
from qbitmap import calibration_data
from qbitmap.noise.simple_noise_model import SimpleNoiseModel
from qbitmap import circuits
from qbitmap import diagnostics

PROJECT ID: fermilab-quantum
PROCESSOR:  >>> rainbow <<<
GATESET:    <cirq_google.serializable_gate_set.SerializableGateSet object at 0x7f5150b2b588>


In [ ]:
from qbitmap import hw

# Don't overwrite any of the analysis using a different processor...!
assert hw.PROCESSOR_ID == "rainbow"

TIMESTAMP = 1628415068308
target_calibration = hw.PROCESSOR.get_calibration(TIMESTAMP // 1000)

print("TIMESTAMP:", TIMESTAMP)

# Target specifically these metrics
metric_1q = "single_qubit_rb_average_error_per_gate"
metric_2q = "two_qubit_sqrt_iswap_gate_xeb_average_error_per_cycle"
calibration = calibration_data.CalibrationWrapper(
    calibration_dct=target_calibration, 
    qubits=hw.DEVICE.qubits, 
    metric_1q=metric_1q, 
    metric_2q=metric_2q
)
fig, ax = plt.subplots(figsize=(10, 10))

calibration.plot_noise_graph(ax=ax)
print()

## Hardware submissions

### Circuit for preparing GHZ state on hardware

In [5]:
template_qubits = cirq.GridQubit.rect(1, 8)
n_qubits = len(template_qubits)

structured_circuit = circuits.ghz_circuit_line_topology_halfdepth(template_qubits, measure=False, native=True)
forward_depth = len(structured_circuit)
structured_circuit_LE = circuits.loschmidt_circuit(circuits.ghz_circuit_line_topology_halfdepth, template_qubits, measure="m")

print("FORWARDS CIRCUIT")
display(structured_circuit)
print("FORWARDS DEPTH=", forward_depth)
print("LE CIRCUIT")
display(structured_circuit_LE)

FORWARDS CIRCUIT


(0, 0): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0.5,x=0.5,z=-0.5)───
                                                                                                                                                                                                                                                          │                                 │
(0, 1): ───PhXZ(a=-1,x=0,z=0.5)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────
                                                                                                                                                                                    │                                 │
(0, 2): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────
                                                                                                              │                                 │
(0, 3): ───PhXZ(a=-1,x=0,z=0.5)───────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                      │                                 │
(0, 4): ───PhXZ(a=-0.75,x=0,z=-0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=-0.5,x=0.5,z=-1)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                              │                                 │
(0, 5): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                    │                                 │
(0, 6): ───PhXZ(a=-1,x=0,z=0.5)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)────────
                                                                                                                                                                                                                                                          │                                 │
(0, 7): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0.5,x=0.5,z=-0.5)───

FORWARDS DEPTH= 17
LE CIRCUIT


(0, 0): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0.5,x=0.5,z=-0.5)───PhXZ(a=0,x=-0.5,z=0.5)───iSwap──────────────────────────────iSwap────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────PhXZ(a=-0.5,x=0,z=-0.5)───M────────
                                                                                                                                                                                                                                                          │                                 │                                                               │                                  │                                                                                                                                                                                                                                                                      │
(0, 1): ───PhXZ(a=-1,x=0,z=0.5)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────PhXZ(a=0,x=-0.5,z=0)─────iSwap^-0.5───PhXZ(a=-1,x=-1,z=0)───iSwap^0.5───PhXZ(a=0.5,x=-0.5,z=-0.5)───iSwap──────────────────────────────iSwap─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────PhXZ(a=-0.5,x=0,z=-0.5)───M────────
                                                                                                                                                                                    │                                 │                                                                                                                                                                                                                │                                  │                                                                                                                                                                                           │
(0, 2): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────PhXZ(a=0,x=-0.5,z=0)────────iSwap^-0.5───PhXZ(a=-1,x=-1,z=0)───iSwap^0.5───PhXZ(a=0.5,x=-0.5,z=-0.5)───iSwap──────────────────────────────iSwap──────────────────────────────────────────────────────────────────────────────────PhXZ(a=-0.5,x=0,z=-0.5)───M────────
                                                                                                              │                                 │                                                                                                                                                                                                                                                                                                                                                                 │                                  │                                                                                                                │
(0, 3): ───PhXZ(a=-1,x=0,z=0.5)───────iSwap─────────────────────────────iSwap───────PhXZ

In [6]:
random_circuit_LE_half = circuits.loschmidt_circuit(
    circuits.create_random_line_circuit, 
    template_qubits, 
    depth=forward_depth // 2, 
    seed=123, 
    measure="m")
print("RANDOM CIRCUIT, depth={} (forward depth)".format(len(random_circuit_LE_half)))
display(random_circuit_LE_half)


RANDOM CIRCUIT, depth=17 (forward depth)


(0, 0): ───PhX(0.5)^0.5──────────────────────PhX(-0.25)^0.5───FSim(0.25π, 0)───PhX(0.75)^0.5─────────────────────PhX(0.5)^0.5─────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0.5)^-0.5───────────────────────PhX(0.75)^-0.5────FSim(-0.25π, 0)───PhX(-0.25)^-0.5─────────────────────PhX(0.5)^-0.5─────M────────
                                                              │                                                                   │                │                                                                       │                                                                       │
(0, 1): ───PhX(0.25)^0.5────FSim(0.25π, 0)───PhX(-0.5)^0.5────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───PhX(0.25)^0.5────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0.25)^-0.5────FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(-0.5)^-0.5────FSim(-0.25π, 0)───PhX(0.25)^-0.5────M────────
                            │                                                                   │                                                                                      │                                                                       │                                   │
(0, 2): ───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(-0.25)^0.5───FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───PhX(0)^0.5───────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0)^-0.5───────FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(-0.25)^-0.5───FSim(-0.25π, 0)───PhX(0.5)^-0.5─────M────────
                                                              │                                                                   │                │                                                                       │                                                                       │
(0, 3): ───PhX(-0.5)^0.5────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(0.5)^-0.5─────FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(-0.5)^-0.5────M────────
                            │                                                                   │                                                                                      │                                                                       │                                   │
(0, 4): ───PhX(0)^0.5───────FSim(0.25π, 0)───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───PhX(0)^0.5───────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0)^-0.5───────FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(0.5)^-0.5─────FSim(-0.25π, 0)───PhX(0)^-0.5───────M────────
                                                              │                                                                   │                │                                                                       │                                                                       │
(0, 5): ───PhX(-0.5)^0.5────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───PhX(0.75)^0.5────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(0.75)^-0.5────FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(-0.5)^-0.5────M────────
                            │                                                                   │                                                                                      │                                                                       │                                   │
(0, 6): ───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───PhX(0.75)^0.5────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0.75)^-0.5────FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(0.5)^-0.5─────M('m')───
                                                              │                                                                   │    

In [10]:
dfe = diagnostics.DirectFidelityEstimator(
        mode="ghz",
        qubits=template_qubits,
        circuit=structured_circuit,
        repetitions=1,
        debug=False,
        readout_calibration=None
    )
#     fid_raw, fid_corr = dfe.run()

In [25]:
dfe_circuits, dfe_params = dfe.dump_estimation_circuit()
print("GHZ DFE CIRCUIT Z-MEASUREMENT")
display(dfe_circuits[0])

print("GHZ DFE CIRCUIT XY-MEASUREMENTS")
for circuit in dfe_circuits[1:]:
    display(circuit)
        
param_dict = dict([(k, []) for k in dfe_params[0].keys()])
for params in dfe_params:
    for (k, v) in params.items():
        param_dict[k].append(v)
print("DFE PARAMETERS FOR XY-MEASUREMENTS")
for (k, v) in param_dict.items():
    print(k, v)


GHZ DFE CIRCUIT Z-MEASUREMENT


(0, 0): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0.5,x=0.5,z=-0.5)───M('m')───
                                                                                                                                                                                                                                                          │                                 │                                      │
(0, 1): ───PhXZ(a=-1,x=0,z=0.5)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────M────────
                                                                                                                                                                                    │                                 │                                                                                                            │
(0, 2): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────M────────
                                                                                                              │                                 │                                                                                                                                                                                  │
(0, 3): ───PhXZ(a=-1,x=0,z=0.5)───────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M────────
                                      │                                 │                                                                                                                                                                                                                                                          │
(0, 4): ───PhXZ(a=-0.75,x=0,z=-0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=-0.5,x=0.5,z=-1)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M────────
                                                                                                              │                                 │                                                                                                                                                                                  │
(0, 5): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────M────────
                                                                                                                                                                                    │                                 │                                                                                                            │
(0, 6): ───PhXZ(a=-1,x=0,z=0.5)─────────────────────

GHZ DFE CIRCUIT XY-MEASUREMENTS


(0, 0): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0.5,x=0.5,z=-0.5)───Rz((0, 0)_local)───H───M('m')───
                                                                                                                                                                                                                                                          │                                 │                                                             │
(0, 1): ───PhXZ(a=-1,x=0,z=0.5)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────Rz((0, 1)_local)───H───M────────
                                                                                                                                                                                    │                                 │                                                                                                                                   │
(0, 2): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────Rz((0, 2)_local)───H───M────────
                                                                                                              │                                 │                                                                                                                                                                                                         │
(0, 3): ───PhXZ(a=-1,x=0,z=0.5)───────iSwap─────────────────────────────iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=0,x=0.5,z=0)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Rz((0, 3)_local)───H───M────────
                                      │                                 │                                                                                                                                                                                                                                                                                 │
(0, 4): ───PhXZ(a=-0.75,x=0,z=-0.5)───iSwap^-0.5───PhXZ(a=-1,x=1,z=0)───iSwap^0.5───PhXZ(a=-0.5,x=0.5,z=-1)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Rz((0, 4)_local)───H───M────────
                                                                                                              │                                 │                                                                                                                                                                                                         │
(0, 5): ───PhXZ(a=-1,x=0,z=0.5)───────────────────────────────────────────────────────────────────────────────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0)──────────────────────────────────────────────────────────────────────────────Rz((0, 5)_local)───H───M────────
                                                                                                                            

DFE PARAMETERS FOR XY-MEASUREMENTS
(0, 0)_local [-0.39269908169872414, -0.7853981633974483, -1.1780972450961724, -1.5707963267948966, -1.9634954084936207, -2.356194490192345, -2.748893571891069, -3.141592653589793]
(0, 1)_local [-0.39269908169872414, -0.7853981633974483, -1.1780972450961724, -1.5707963267948966, -1.9634954084936207, -2.356194490192345, -2.748893571891069, -3.141592653589793]
(0, 2)_local [-0.39269908169872414, -0.7853981633974483, -1.1780972450961724, -1.5707963267948966, -1.9634954084936207, -2.356194490192345, -2.748893571891069, -3.141592653589793]
(0, 3)_local [-0.39269908169872414, -0.7853981633974483, -1.1780972450961724, -1.5707963267948966, -1.9634954084936207, -2.356194490192345, -2.748893571891069, -3.141592653589793]
(0, 4)_local [-0.39269908169872414, -0.7853981633974483, -1.1780972450961724, -1.5707963267948966, -1.9634954084936207, -2.356194490192345, -2.748893571891069, -3.141592653589793]
(0, 5)_local [-0.39269908169872414, -0.7853981633974483, -1.17809

In [ ]:
noise_graph = calibration.noise_graph
# Pre-compute all of the paths
all_paths = []
for source in noise_graph.nodes():
    for target in noise_graph.nodes():
        if source == target:
            continue            
        all_paths_ij = list(nx.all_simple_paths(noise_graph, source, target, cutoff=n_qubits))
        for v in all_paths_ij:
            if len(v) == n_qubits:
                all_paths.append(v)

print("Found {} paths with {} qubits to attempt".format(len(all_paths), n_qubits))

#### Issue

A notebook cell was inadvertently deleted during this reservation. This is a problem because this experiment involved "stitching" batches of experiments together from between interruptions in the QCS availability. Typically, stitching is done using hard-coded indices that indicate when the interruption occured and thus (a) which experiments to discard due to corruption from the interruption and (b) how to synchronize results for different metrics to the same set of indices.

I attempted to recover the source code from a previous branch which should correspond to exactly what ran at the experiment date. However, later stitching operations appear to have failed suggesting that an important index was permanently lost in the above procedure. The final ~80 experiments in this batch have been discarded as their stitch index could not be verified.

### EXPERIMENT COMPLETE. RUNNING WILL OVERWRITE DATA

In [ ]:
 # Dry run tests: 5 passes finished in 5:05 minutes
# = ~1 minute per full experiment


# Set DRY_RUN to dump readout ec metadata into a different folder
DRY_RUN = False
def dry_run_print(s, dry_run=DRY_RUN):
    if dry_run:
        print(s)
DATESTR = "20210809"

bf_timestamp = str(TIMESTAMP)
readout_ec_path = "./readout_ec"
if DRY_RUN:
    readout_ec_path = "./dryrun"
    bf_timestamp = str(TIMESTAMP) + str(np.random.random() * 1000)
    
DIAGNOSTIC_REPS = 10_000
REPETITIONS = 15_000
N_EXPERIMENTS = 350

# results[:,0] stores F_LE, results[:,1] stores F, results[:,2] stores F0
# Raw refers to no readout error correction, corr refers to yes readout EC
results_raw = np.zeros((N_EXPERIMENTS, 3))
results_corr = np.zeros((N_EXPERIMENTS, 3))

# Configuration for random circuits
N_TRIALS = 5 # Number of random circuits to attempt per qubit configuration
# For each random circuit, we'll attempt one with(forward depth) and one for 2*(forward depth)
random_results_raw = np.zeros((N_EXPERIMENTS, N_TRIALS))
random_results_corr = np.zeros((N_EXPERIMENTS, N_TRIALS))
seeds = [3882, 175, 802, 993, 22]
assert len(seeds) >= N_TRIALS

used_paths = []

i_RESTART = 300
for i in range(N_EXPERIMENTS):
    if i < i_RESTART:
        continue
    if (i % 5) == 0:
        print(f"run={i}")
        
    # Construct the circuit on this path
    v = shuffled_paths[i]
    targets = [cirq.GridQubit(*x) for x in v]
    qubit_map = dict(zip(template_qubits, targets))   
    used_paths.append(v)
    
    # Perform separable readout error diagnostic
    # Every iteration needs a unique identifier for its readout error diagnostic
    hw_diagnostic_sep = diagnostics.SeparableReadoutErrorDiagnostic(
        timestamp=bf_timestamp + f"_{i}" ,
        qubits=targets,
        repetitions=DIAGNOSTIC_REPS,
        debug=False,
        path=readout_ec_path
    )
    qvals = hw_diagnostic_sep.run(ntrials=1)
    dry_run_print("...readout error diagnostic complete")

#     Compute structured circuit loschmidt survival
#     Don't overwrite the structured LE circuit
    mapped_LE_circuit = structured_circuit_LE.transform_qubits(qubit_map)
    job = hw.ENGINE.run_sweep(
        program=mapped_LE_circuit,
        repetitions=REPETITIONS,
        processor_ids=[hw.PROCESSOR_ID],
        gate_set=hw.GATESET
    )
    # F_LE computation
    LE_counter = job.results()[0].histogram(key="m")
    results_raw[i,0] = LE_counter.get(0) / REPETITIONS
    LE_arr = utils.hist_as_np(LE_counter, n_qubits, REPETITIONS)
    results_corr[i,0] = hw_diagnostic_sep.invert_and_correct(LE_arr)[0]
    dry_run_print("...F_LE complete")

    # Perform Direct Fidelity Estimation with and without readout correction
    mapped_forward_circuit = structured_circuit.transform_qubits(qubit_map)
    dfe = diagnostics.DirectFidelityEstimator(
        mode="ghz",
        qubits=targets,
        circuit=mapped_forward_circuit,
        repetitions=REPETITIONS,
        debug=False,
        readout_calibration=hw_diagnostic_sep
    )
    fid_raw, fid_corr = dfe.run()
    results_corr[i,1] = fid_corr
    results_raw[i,1] = fid_raw
    dry_run_print("...F DFE complete")

    # Perform random circuit runs with readout error correction
    rand_fwd_depth = forward_depth // 2 
    # iterate over random circuit depths
    for j in range(N_TRIALS):
        # Iterate over rand
        random_circuit_LE = circuits.loschmidt_circuit(
            circuits.create_random_line_circuit, 
            template_qubits, 
            depth=rand_fwd_depth, 
            seed=seeds[j] + i, 
            measure="m")
        dry_run_print("rand circuit LE depth={}".format(len(random_circuit_LE)))
        random_circuit_LE = random_circuit_LE.transform_qubits(qubit_map)
        # Submit hardware random circuit
        job = hw.ENGINE.run_sweep(
            program=random_circuit_LE,
            repetitions=REPETITIONS,
            processor_ids=[hw.PROCESSOR_ID],
            gate_set=hw.GATESET
        )
        # raw and corr
        random_counter = job.results()[0].histogram(key="m")
        random_results_raw[i, j] = random_counter.get(0) / REPETITIONS
        rand_arr = utils.hist_as_np(random_counter, n_qubits, REPETITIONS)
        random_results_corr[i,j] = hw_diagnostic_sep.invert_and_correct(rand_arr)[0]
    dry_run_print("...Random F_LE complete")

        
    # Perform zeroth order metric with and without readout error correction assumption
    F0 = metrics.compute_calibration_fidelity(mapped_forward_circuit, noise_graph)
    F0_readout_err = metrics.compute_calibration_fidelity(mapped_forward_circuit, noise_graph, readout_error=True)
    results_corr[i,2] = F0
    results_raw[i,2] = F0_readout_err
    
    if DRY_RUN and i == 10:
        break
    # Caching
    np.save(f"./temp/v6_hw_{DATESTR}_ghz_line_results_corr_{i}.npy", results_corr)
    np.save(f"./temp/v6_hw_{DATESTR}_ghz_line_results_raw_{i}.npy", results_raw)
    np.save(f"./temp/v6_hw_{DATESTR}_random_line_results_raw_{i}.npy", random_results_raw)
    np.save(f"./temp/v6_hw_{DATESTR}_random_line_results_corr_{i}.npy", random_results_corr)
    np.save(f"./temp/v6_hw_{DATESTR}_ghz_line_vs_random_paths_{i}.npy", np.asarray(used_paths))

if not DRY_RUN:
    np.save(f"v6_hw_{DATESTR}_ghz_line_results_corr_{i_RESTART}_tofinish.npy", results_corr)
    np.save(f"v6_hw_{DATESTR}_ghz_line_results_raw_{i_RESTART}_tofinish.npy", results_raw)
    np.save(f"v6_hw_{DATESTR}_random_line_results_raw_{i_RESTART}_tofinish.npy", random_results_raw)
    np.save(f"v6_hw_{DATESTR}_random_line_results_corr_{i_RESTART}_tofinish.npy", random_results_corr)
    np.save(f"v6_hw_{DATESTR}_ghz_line_vs_random_paths_{i_RESTART}_tofinish.npy", np.asarray(used_paths))

### Stitching operations

In [ ]:
import numpy as np
DATESTR = "20210809"

x = np.load(f"v6_hw_{DATESTR}_ghz_line_results_corr_273.npy")[:274]
y = np.load(f"v6_hw_{DATESTR}_ghz_line_results_corr_274_tofinish.npy")[274:]
z = np.load(f"v6_hw_{DATESTR}_ghz_line_results_corr_300_tofinish.npy")[300:]
np.save(f"./results/v6_hw_{DATESTR}_ghz_line_results_corr.npy", np.vstack((x,y,z)))

x2 = np.load(f"v6_hw_{DATESTR}_ghz_line_results_raw_273.npy")[:274]
y2 = np.load(f"v6_hw_{DATESTR}_ghz_line_results_raw_274_tofinish.npy")[274:]
z2 = np.load(f"v6_hw_{DATESTR}_ghz_line_results_raw_300_tofinish.npy")[300:]
np.save(f"./results/v6_hw_{DATESTR}_ghz_line_results_raw.npy", np.vstack((x2,y2,z2)))

x3 = np.load(f"v6_hw_{DATESTR}_random_line_results_raw_273.npy")[:274]
y3 = np.load(f"v6_hw_{DATESTR}_random_line_results_raw_274_tofinish.npy")[274:]
z3 = np.load(f"v6_hw_{DATESTR}_random_line_results_raw_300_tofinish.npy")[300:]
np.save(f"./results/v6_hw_{DATESTR}_random_line_results_raw.npy", np.vstack((x3,y3,z3)))

x4 = np.load(f"v6_hw_{DATESTR}_random_line_results_corr_273.npy")[:274]
y4 = np.load(f"v6_hw_{DATESTR}_random_line_results_corr_274_tofinish.npy")[274:]
z4 = np.load(f"v6_hw_{DATESTR}_random_line_results_corr_300_tofinish.npy")[300:]
np.save(f"./results/v6_hw_{DATESTR}_random_line_results_corr.npy", np.vstack((x4,y4,z4)))

# CANNOT VERIFY WHAT HAPPENED WITH FINAL BATCH
x5 = np.load(f"v6_hw_{DATESTR}_ghz_line_vs_random_paths_273.npy")[:274]
y5 = np.load(f"v6_hw_{DATESTR}_ghz_line_vs_random_paths_274_tofinish.npy")[274:]
z5 = np.load(f"v6_hw_{DATESTR}_ghz_line_vs_random_paths_300_tofinish.npy")[300:]

